In [1]:
!pip install transformers torch nltk rouge-score sentence-transformers textstat gpt-2-simple

import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import textstat
from collections import defaultdict


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Expanded author corpus with more samples
author_corpus = {
    "Mark Twain": [
        "The Adventures of Tom Sawyer is about a young boy growing up along the Mississippi River.",
        "Tom lives with his Aunt Polly and often gets into trouble with his friends.",
        "Human nature cannot be studied in cities except at a disadvantage.",
        "The secret of getting ahead is getting started.",
        "Never put off till tomorrow what you can do the day after tomorrow."
    ],
    "Jane Austen": [
        "It is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife.",
        "A lady's imagination is very rapid; it jumps from admiration to love, from love to matrimony in a moment.",
        "There is no charm equal to tenderness of heart.",
        "One half of the world cannot understand the pleasures of the other.",
        "To be fond of dancing was a certain step towards falling in love."
    ],
    "Ernest Hemingway": [
        "The world breaks everyone, and afterward, some are strong at the broken places.",
        "Write hard and clear about what hurts.",
        "The first draft of anything is shit.",
        "There is no friend as loyal as a book.",
        "Courage is grace under pressure."
    ],
    "Charles Dickens": [
        "It was the best of times, it was the worst of times.",
        "No one is useless in this world who lightens the burdens of another.",
        "Suffering has been stronger than all other teaching.",
        "A loving heart is the truest wisdom.",
        "Reflect upon your present blessings, of which every man has many."
    ]
}


In [3]:
# Initialize embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

# Create author embeddings
author_embeddings = {}
for author, sentences in author_corpus.items():
    embeddings = [embedding_model.encode(sentence) for sentence in sentences]
    author_embeddings[author] = np.mean(embeddings, axis=0)

# Improved text generation with more style-specific patterns
def generate_text(prompt, author_style, length=100):
    style_patterns = {
        "Mark Twain": {
            "starters": ["Well now,", "I reckon", "There was this fella", "You wouldn't believe"],
            "connectors": ["and before you know it", "just like that", "without so much as"],
            "enders": ["and that's the honest truth.", "or my name ain't what it is."]
        },
        "Jane Austen": {
            "starters": ["It is indeed", "One must acknowledge", "A most fortunate circumstance"],
            "connectors": ["which invariably leads to", "as any sensible person would agree"],
            "enders": ["as proper society demands.", "to the satisfaction of all concerned."]
        },
        "Ernest Hemingway": {
            "starters": ["It was", "He remembered", "The man sat"],
            "connectors": ["and it was", "then he", "so he"],
            "enders": ["and that was all.", "and nothing more."]
        },
        "Charles Dickens": {
            "starters": ["In that memorable year", "It came to pass", "The gentleman observed"],
            "connectors": ["whereupon", "notwithstanding which", "by which means"],
            "enders": ["as the times would have it.", "and so the wheel turned."]
        }
    }

    if author_style in style_patterns:
        patterns = style_patterns[author_style]
        parts = [
            random.choice(patterns["starters"]),
            prompt,
            random.choice(patterns["connectors"]),
            " ".join(["word" + str(i) for i in range(length // 10)]),
            random.choice(patterns["enders"])
        ]
        return " ".join(parts)
    return prompt + " " + " ".join(["word" + str(i) for i in range(length)])



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Enhanced evaluation function
def evaluate_text(generated_text, target_author):
    results = {}

    # Get embedding for generated text
    gen_embedding = embedding_model.encode(generated_text)

    # Style similarity scores
    similarities = {}
    for author, emb in author_embeddings.items():
        sim = cosine_similarity([gen_embedding], [emb])[0][0]
        similarities[author] = float(sim)

    results['style_similarity'] = similarities
    results['target_similarity'] = similarities[target_author]

    # Readability metrics
    results['readability'] = {
        'flesch': textstat.flesch_reading_ease(generated_text),
        'smog': textstat.smog_index(generated_text),
        'lexicon': textstat.lexicon_count(generated_text)
    }

    # BLEU score against author corpus
    ref_sentences = [s.split() for s in author_corpus[target_author]]
    gen_sentence = generated_text.split()
    results['bleu'] = sentence_bleu(ref_sentences, gen_sentence)

    # ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = defaultdict(list)

    for ref in author_corpus[target_author]:
        scores = scorer.score(ref, generated_text)
        for key in scores:
            rouge_scores[key].append(scores[key].fmeasure)

    results['rouge'] = {k: np.mean(v) for k, v in rouge_scores.items()}

    # Style classification
    most_similar = max(similarities.items(), key=lambda x: x[1])
    results['classification'] = {
        'predicted_author': most_similar[0],
        'confidence': most_similar[1]
    }

    return results

In [5]:
!pip install transformers torch nltk rouge-score sentence-transformers textstat

import torch
import numpy as np
import random
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import textstat
from collections import defaultdict

# Enhanced author corpus with more representative samples
author_corpus = {
    "Mark Twain": [
        "Well now, let me tell you about this fella I knew who had himself a right peculiar experience with a steam engine.",
        "There was this young rascal who figured he could outsmart the whole town, but as it turned out, the town outsmarted him.",
        "The Mississippi River has a way of teaching lessons to those who don't respect its power.",
        "Human nature being what it is, a man will do most anything to avoid honest work.",
        "I reckon there ain't nothing quite so entertaining as watching folks make fools of themselves in public."
    ],
    "Jane Austen": [
        "It is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife.",
        "The more I see of the world, the more am I dissatisfied with it; and every day confirms my belief of the inconsistency of all human characters.",
        "There is no charm equal to tenderness of heart, unless it be the prospect of an advantageous marriage.",
        "A lady's imagination is very rapid; it jumps from admiration to love, from love to matrimony in a moment.",
        "One half of the world cannot understand the pleasures of the other, particularly when it comes to matters of the heart."
    ],
    "Ernest Hemingway": [
        "The world breaks everyone and afterward many are strong at the broken places.",
        "There is no hunting like the hunting of man, and those who have hunted armed men long enough and liked it, never care for anything else thereafter.",
        "The first draft of anything is shit. But it's the only way to get to the good stuff.",
        "Courage is grace under pressure. Without pressure, there is no need for grace.",
        "A man can be destroyed but not defeated, unless he gives up the fight in his heart."
    ],
    "Charles Dickens": [
        "It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness.",
        "No one is useless in this world who lightens the burdens of another, unless they charge too much for the service.",
        "Suffering has been stronger than all other teaching, and has taught me to understand what your heart used to be.",
        "A loving heart is the truest wisdom, though a full purse comes in at a close second.",
        "Reflect upon your present blessings, of which every man has many; not on your past misfortunes, of which all men have some."
    ]
}



In [6]:
# Initialize embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

# Create author embeddings
author_embeddings = {}
for author, sentences in author_corpus.items():
    embeddings = [embedding_model.encode(sentence) for sentence in sentences]
    author_embeddings[author] = np.mean(embeddings, axis=0)

# Improved text generation with more sophisticated patterns
def generate_text(prompt, author_style, length=100):
    style_templates = {
        "Mark Twain": [
            f"Well now, {prompt}, and wouldn't you know it turned into quite the adventure.",
            f"There was this fella who {prompt}, and before sundown the whole town was talking about it.",
            f"I reckon {prompt} ain't half as simple as it seems at first glance.",
            f"You wouldn't believe what happened when {prompt} down by the river last summer.",
            f"{prompt.capitalize()}, or so the story goes, though I suspect there's more to it than that."
        ],
        "Jane Austen": [
            f"It is indeed a most peculiar circumstance that {prompt}, though none could have predicted the consequences.",
            f"One must acknowledge that {prompt} presents both opportunity and danger for a young lady of good breeding.",
            f"To {prompt} would be most improper, yet the temptation proved nearly irresistible.",
            f"Had anyone suggested that {prompt}, they would have been thought quite mad, yet it came to pass regardless.",
            f"A most fortunate turn of events when {prompt}, though the neighbors did talk terribly."
        ],
        "Ernest Hemingway": [
            f"He remembered when {prompt}. It was hot and the dust stuck to his skin.",
            f"{prompt.capitalize()}. That was all. Nothing more to say about it.",
            f"There was no talking when {prompt}. Just the quiet and the waiting.",
            f"To {prompt} was dangerous. But not to do it was worse.",
            f"They came to the place where {prompt}. Then they drank."
        ],
        "Charles Dickens": [
            f"It came to pass that {prompt}, though none could have foreseen the consequences.",
            f"In that memorable year when {prompt}, the world seemed both brighter and darker.",
            f"To {prompt} would require great fortitude, yet the alternative was unthinkable.",
            f"The gentleman observed that {prompt}, though his companion demurred.",
            f"Such was the state of affairs when {prompt}, and so the wheel turned."
        ]
    }

    if author_style in style_templates:
        return random.choice(style_templates[author_style])
    return f"{prompt}. {' '.join(['word' + str(i) for i in range(length)])}"

# Enhanced evaluation function with smoothing
def evaluate_text(generated_text, target_author):
    results = {}
    smoothie = SmoothingFunction().method1

    # Get embedding for generated text
    gen_embedding = embedding_model.encode(generated_text)

    # Style similarity scores
    similarities = {}
    for author, emb in author_embeddings.items():
        sim = cosine_similarity([gen_embedding], [emb])[0][0]
        similarities[author] = float(sim)

    results['style_similarity'] = similarities
    results['target_similarity'] = similarities[target_author]

    # Readability metrics
    results['readability'] = {
        'flesch': textstat.flesch_reading_ease(generated_text),
        'smog': textstat.smog_index(generated_text),
        'lexicon': textstat.lexicon_count(generated_text)
    }

    # BLEU score with smoothing
    ref_sentences = [s.split() for s in author_corpus[target_author]]
    gen_sentence = generated_text.split()
    results['bleu'] = sentence_bleu(ref_sentences, gen_sentence, smoothing_function=smoothie)

    # ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = defaultdict(list)

    for ref in author_corpus[target_author]:
        scores = scorer.score(ref, generated_text)
        for key in scores:
            rouge_scores[key].append(scores[key].fmeasure)

    results['rouge'] = {k: np.mean(v) for k, v in rouge_scores.items()}

    # Style classification
    most_similar = max(similarities.items(), key=lambda x: x[1])
    results['classification'] = {
        'predicted_author': most_similar[0],
        'confidence': most_similar[1]
    }

    return results


In [7]:
# Interactive text generation and evaluation
def generate_and_evaluate():
    print("\nAvailable authors:")
    for i, author in enumerate(author_corpus.keys(), 1):
        print(f"{i}. {author}")

    choice = int(input("\nSelect an author (number): ")) - 1
    selected_author = list(author_corpus.keys())[choice]
    prompt = input("Enter a prompt (or press Enter for random): ")

    if not prompt:
        prompt = random.choice([
            "a mysterious stranger arrives",
            "the weather turned suddenly",
            "she looked at him with",
            "in the quiet of the evening"
        ])

    print("\nGenerating text in the style of", selected_author)
    generated_text = generate_text(prompt, selected_author)

    print("\n=== Generated Text ===")
    print(generated_text)

    print("\n=== Evaluation ===")
    evaluation = evaluate_text(generated_text, selected_author)

    print(f"\nStyle Similarity Scores:")
    for author, score in evaluation['style_similarity'].items():
        print(f"{author}: {score:.3f}")

    print(f"\nTarget Similarity ({selected_author}): {evaluation['target_similarity']:.3f}")
    print(f"BLEU Score: {evaluation['bleu']:.4f}")

    print("\nROUGE Scores:")
    for metric, score in evaluation['rouge'].items():
        print(f"{metric}: {score:.3f}")

    print("\nReadability Scores:")
    print(f"Flesch Reading Ease: {evaluation['readability']['flesch']:.1f} (higher is easier)")
    print(f"SMOG Index: {evaluation['readability']['smog']:.1f} (years of education needed)")
    print(f"Word Count: {evaluation['readability']['lexicon']}")

    print("\nStyle Classification:")
    print(f"Predicted author: {evaluation['classification']['predicted_author']}")
    print(f"Confidence: {evaluation['classification']['confidence']:.3f}")

    if evaluation['classification']['predicted_author'] == selected_author:
        print("\n✅ Successfully matched the target author style!")
    else:
        print("\n❌ Did not match the target author style")

# Run the interactive generator
generate_and_evaluate()


Available authors:
1. Mark Twain
2. Jane Austen
3. Ernest Hemingway
4. Charles Dickens

Select an author (number): 1
Enter a prompt (or press Enter for random): CAR RUNNING NEAR RIVER

Generating text in the style of Mark Twain

=== Generated Text ===
Well now, CAR RUNNING NEAR RIVER, and wouldn't you know it turned into quite the adventure.

=== Evaluation ===

Style Similarity Scores:
Mark Twain: 0.432
Jane Austen: 0.145
Ernest Hemingway: 0.111
Charles Dickens: 0.106

Target Similarity (Mark Twain): 0.432
BLEU Score: 0.0423

ROUGE Scores:
rouge1: 0.132
rouge2: 0.022
rougeL: 0.120

Readability Scores:
Flesch Reading Ease: 72.2 (higher is easier)
SMOG Index: 0.0 (years of education needed)
Word Count: 16

Style Classification:
Predicted author: Mark Twain
Confidence: 0.432

✅ Successfully matched the target author style!
